In [1]:
import os
os.chdir('E:\\')

In [134]:
import requests
from requests_html import HTML
import json
import numpy as np
import pandas as pd
import time

base_url = 'https://stackoverflow.com/questions'
tag = 'python'
query_filter = 'Votes'
url = f"{base_url}"
url

'https://stackoverflow.com/questions'

In [135]:
r = requests.get(url)
html_str = r.text

In [136]:
html = HTML(html=html_str)

In [137]:
question_summaries = html.find(".s-post-summary") 
print(question_summaries)

[<Element 'div' id='question-summary-76546165' class=('s-post-summary', 'js-post-summary') data-post-id='76546165' data-post-type-id='1'>, <Element 'div' id='question-summary-76546164' class=('s-post-summary', 'js-post-summary') data-post-id='76546164' data-post-type-id='1'>, <Element 'div' id='question-summary-76546162' class=('s-post-summary', 'js-post-summary') data-post-id='76546162' data-post-type-id='1'>, <Element 'div' id='question-summary-76546158' class=('s-post-summary', 'js-post-summary') data-post-id='76546158' data-post-type-id='1'>, <Element 'div' id='question-summary-76546156' class=('s-post-summary', 'js-post-summary') data-post-id='76546156' data-post-type-id='1'>, <Element 'div' id='question-summary-76546155' class=('s-post-summary', 'js-post-summary') data-post-id='76546155' data-post-type-id='1'>, <Element 'div' id='question-summary-76546154' class=('s-post-summary', 'js-post-summary') data-post-id='76546154' data-post-type-id='1'>, <Element 'div' id='question-summa

In [138]:
question_summaries[0].text

"0 votes\n0 answers\n2 views\nWhich decision tree algorithm does scikit-learn use when the criterion is 'enropy'?\nAs the title,which decision tree algorithm does scikit-learn use when the criterion is 'enropy'? In the doc,it writes CART,but in fact CART uses gini. It confused me a lot I compared gini and entropy ...\nscikit-learn\ndt\ncwl142536\n1\nasked 1 min ago"

In [141]:
this_row =list(question_summaries[10].text.split('\n'))
this_row

['0 votes',
 '0 answers',
 '4 views',
 'How to convert first audio track only to all mkv with ffmpeg',
 'I am currently using the following code in terminal: for i in ./*.mkv; do ffmpeg -i "$i" -map 0:v -map 0:a -map 0:s -c:v copy -c:a libopus -c:s copy -vbr 0 -b:a 256k "${i%????}_muxed....',
 'ffmpeg',
 'terminal',
 'encode',
 'gorilla',
 'Not your kind of people',
 '120',
 'asked 6 mins ago']

In [ ]:
key_names = ['user', 'question','votes','tags','date']
class_needed = ['.s-user-card--link','.s-link','.s-post-summary--stats-item-number','.s-post-summary--meta-tags','.relativetime']

In [ ]:
def clean_data(text, keyname=None):
    if keyname == 'tags':
        return text.replace('\n', ' & ')
    return text

In [ ]:
def parse_tagged_page(html):
    question_summaries = html.find(".s-post-summary") 
    key_names = ['user', 'question','votes','date']
    class_needed = ['.s-user-card--link','.s-link','.s-post-summary--stats-item-number','.relativetime']
    data = []

    for items in question_summaries:
        q_data = {}
        for i, elements in enumerate(class_needed):
            sub_elements = items.find(elements, first = True)
            keyname = key_names[i]
            if sub_elements:
                q_data[keyname]=clean_data(sub_elements.text, keyname=keyname)
            else:
                q_data[keyname]=clean_data(np.nan, keyname=keyname)
        data.append(q_data)
    return data

In [ ]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    data = parse_tagged_page(html)
    return data

In [ ]:
def scrape_tag(tag = "python", query_filter = "Votes", max_pages=100, pagesize=25):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    data = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        data += extract_data_from_url(url)
        time.sleep(1.3)
    return data

In [ ]:
data = scrape_tag(tag='python')

In [ ]:
len(data)

In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv("stk_api_scrap.csv", index=False)